In [16]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder


In [17]:
class KSOM(nn.Module):
    def __init__(self, input_size, output_size, sigma=1.0):
        super(KSOM, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.sigma = sigma
        self.weights = nn.Parameter(torch.randn(output_size, input_size) * 0.01)

    def forward(self, x):
        dist_to_winner = torch.cdist(x, self.weights)
        neighbors = torch.exp(-dist_to_winner / (2 * self.sigma ** 2))
        return neighbors


In [18]:
train_dir = 'car_data/train'
test_dir = 'car_data/test'

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])


In [19]:
train_dataset = ImageFolder(root=train_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = ImageFolder(root=test_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

input_size = 3 * 224 * 224
num_classes = len(train_dataset.classes)
output_size = num_classes
sigma = 1.0

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [21]:
som = KSOM(input_size, output_size, sigma).to(device)

epochs = 20
lr = 0.1
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(som.parameters(), lr=lr)

In [22]:
for epoch in range(epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        images = images.view(-1, input_size).to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = som(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

Epoch 1, Loss: 5.278116215020418
Epoch 2, Loss: 5.278116215020418
Epoch 3, Loss: 5.278116215020418
Epoch 4, Loss: 5.278116215020418
Epoch 5, Loss: 5.278116215020418
Epoch 6, Loss: 5.278116215020418
Epoch 7, Loss: 5.278116215020418
Epoch 8, Loss: 5.278116215020418
Epoch 9, Loss: 5.278116215020418
Epoch 10, Loss: 5.278116215020418
Epoch 11, Loss: 5.278116215020418
Epoch 12, Loss: 5.278116215020418
Epoch 13, Loss: 5.278116215020418
Epoch 14, Loss: 5.278116215020418
Epoch 15, Loss: 5.278116215020418
Epoch 16, Loss: 5.278116215020418
Epoch 17, Loss: 5.278116215020418
Epoch 18, Loss: 5.278116215020418
Epoch 19, Loss: 5.278116215020418
Epoch 20, Loss: 5.278116215020418


In [24]:
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.view(-1, input_size).to(device)
        neighbors = som(images)
        _, predicted_indices = torch.max(neighbors, 1)
        for i in range(len(predicted_indices)):
            predicted_cluster = train_dataset.classes[predicted_indices[i].item()]
            ground_truth_cluster = train_dataset.classes[labels[i].item()]
            if predicted_cluster == ground_truth_cluster:
                correct += 1
            total += 1

accuracy = correct / total
print("Accuracy:", accuracy)

Accuracy: 0.006466857356050243
